In [9]:
# load api keys from the `.env` file. An example of this can be found in `.env.sample`
from dotenv import load_dotenv; load_dotenv()

True

In [3]:
# Get Data
#import csv

from datasets import load_dataset
ds = load_dataset("stanfordnlp/sentiment140")


Generating test split: 100%|██████████| 498/498 [00:00<00:00, 30145.67 examples/s]


In [4]:
docs = ds['train'][:2000]['text']

In [5]:
# Get our encoder to encode 
from sentence_transformers import SentenceTransformer 
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
data_emb = model.encode(docs) #16 seconds on M1 Mac 8gb


/Users/ksolo/Library/Caches/pypoetry/virtualenvs/aimodels-HUywTnIy-py3.12/lib/python3.12/site-packages/sentence_transformers/evaluation/SentenceEvaluator.py:81: SyntaxWarning: invalid escape sequence '\g'
  return re.sub(r"([a-z])([A-Z])", "\g<1> \g<2>", class_name)
/Users/ksolo/Library/Caches/pypoetry/virtualenvs/aimodels-HUywTnIy-py3.12/lib/python3.12/site-packages/sentence_transformers/model_card.py:524: SyntaxWarning: invalid escape sequence '\d'
  if dataset_name and re.match("_dataset_\d+", dataset_name):
/Users/ksolo/Library/Caches/pypoetry/virtualenvs/aimodels-HUywTnIy-py3.12/lib/python3.12/site-packages/sentence_transformers/losses/DenoisingAutoEncoderLoss.py:16: SyntaxWarning: invalid escape sequence '\_'
  """


In [6]:
# Now set up the vector store to accept the data

import chromadb

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="SampleDB")

collection.add(
    embeddings=data_emb.tolist(),
    documents=docs,
    ids=[str(idx) for idx in range(len(data_emb))]) # Doc ID's are required



In [7]:
#prep the question in the encoding space

question = 'What is the status of the chalkboard?'
question_emb = model.encode(question)

results = collection.query(query_embeddings=question_emb.tolist(), n_results=10)

context = ' '.join(results['documents'][0]) # Pulling out a lists of lists


In [8]:
# Call the models to determine the answer by response

prompt = f'Given the following data, Please answer the question:  \n\n ##question \n {question}\n\n ##context \n {context}'

import aimodels as ai
client = ai.Client()

messages = [
    {"role": "system", "content": "You are a helpful agent, who answers with brevity. "},
    {"role": "user", "content": prompt},
]


In [10]:
#groq_llama3_8b = "groq:llama3-8b-8192" 
response = client.chat.completions.create(model="groq:llama3-70b-8192", messages=messages)

print(response.choices[0].message.content)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The status of the chalkboard is: USELESS (because there is no chalk).


In [11]:
results['documents'][0][0]

"Damn... I don't have any chalk! MY CHALKBOARD IS USELESS "

In [12]:
response = client.chat.completions.create(model="anthropic:claude-3-opus-20240229", messages=messages)
print(response.choices[0].message.content)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Based on the context provided, the status of the chalkboard is useless because the person does not have any chalk.
